In [57]:
import sys
import json
import csv
import os

import pandas as pd
import numpy as np

# import matplotlib as mpl

# useful for printing dict and list objects
import pprint

import time
from datetime import datetime

import psycopg2
from sqlalchemy import create_engine, text as sql_text
import psycopg2.extras
from dotenv import load_dotenv



In [58]:
load_dotenv()

#get user input
db_username = os.environ['db_username']
db_password = os.environ['db_password']
                             
db_eng = create_engine('postgresql+psycopg2://' + db_username + ':' + db_password + '@localhost:5432/airbnb',
                       connect_args={'options': '-csearch_path={}'.format('airbnb')},
                       isolation_level = 'SERIALIZABLE')

print("Successfully created db engine.")


Successfully created db engine.


In [59]:
tags = ["__", "__datetime_in_reviews__", "__id_in_listings__", "__datetime_in_reviews__id_in_listings__"]

In [60]:
filename = 'listings_join_reviews.json'

if os.path.exists(filename):
    try:
        # delete the file
        os.remove(filename)
        print(f"File '{filename}' successfully deleted.")
    except Exception as e:
        print(f"Error deleting file '{filename}': {e}")
else:
    print(f"File '{filename}' not found.")

#write perf data to file
def write_perf_data(year, tag, result, save_path= filename):
    """
    - year: year to search on
    - result: dict，stats from the query
    - save_path: JSON file path to save the results
    """
    year_key = f"listing_join_reviews_{year}"

    if os.path.exists(save_path):
        with open(save_path, "r") as f:
            existing = json.load(f)
    else:
        existing = {}

    if year_key not in existing:
        existing[year_key] = {}

    existing[year_key][tag] = result

    with open(save_path, "w") as f:
        json.dump(existing, f, indent=4)

    print(f"successfully wrote {tag} {year} to {save_path}")
    print("\n")


File 'listings_join_reviews.json' successfully deleted.


In [61]:
# function to get the query for the year
def yyyy_query(year):
    year_str = str(year)
    start_date = f"{year_str}-01-01"
    end_date = f"{year_str}-12-31"

    query = f"""
    SELECT * 
    FROM listings l, reviews r 
    WHERE l.id = r.listing_id
      AND r.datetime >= '{start_date}'
      AND r.datetime <= '{end_date}'
    ORDER BY l.id;
    """
    
    return query


#run time
n= 50

#check indexes
q_show_indexes = '''
select *
from pg_indexes
WHERE tablename IN ('reviews', 'listings');
'''

### combination 1
### no indexes

In [62]:
#Add index on `reviews.datetime`
q_drop_datetime_index_in_reviews = '''
DROP INDEX IF EXISTS datetime_in_reviews;
'''

#Remove index on `listings.id`
q_drop_id_in_listings = '''
DROP INDEX IF EXISTS id_in_listings;
'''

In [63]:
def no_index(q, year):

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_drop_datetime_index_in_reviews))

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_drop_id_in_listings))

    # delete indexes if they exist
    with db_eng.begin() as conn:
        result_reviews = conn.execute(sql_text(q_show_indexes))
        print()
        print('The set of indexes on reviews and listings is: ')
        print(result_reviews.all())


    # run it 50 times
    times = []
    for i in range(n):
        start = time.time()
        with db_eng.begin() as conn:
            conn.execute(sql_text(q)).fetchall()
        end = time.time()
        times.append(end - start)

    #store results
    return {
                'avg': round(sum(times) / len(times), 4),
                'min': round(min(times), 4),
                'max': round(max(times), 4),
                'std': round(np.std(times), 4),
                'count': len(times),
                'timestamp': datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
    }


### combination 2
### datetime_in_reviews index exists

In [64]:
#create index on reviews.datetime
q_create_datetime_index_in_reviews = '''
CREATE INDEX IF NOT EXISTS datetime_in_reviews
ON reviews(datetime);
'''

#delete the index on listings.id if it exists
q_drop_id_in_listings = '''
DROP INDEX IF EXISTS id_in_listings;
'''

In [65]:
def index_datetime_in_reviews(q, year):

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_create_datetime_index_in_reviews))

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_drop_id_in_listings))

    #execute the queries to create the index and drop the id_in_listings index and check
    with db_eng.begin() as conn:
        result_reviews = conn.execute(sql_text(q_show_indexes))
        print()
        print('The set of indexes on reviews and listings is: ')
        print(result_reviews.all())


    # run it 50 times
    times = []
    for i in range(n):
        start = time.time()
        with db_eng.begin() as conn:
            conn.execute(sql_text(q)).fetchall()
        end = time.time()
        times.append(end - start)

    #store results
    return {
                'avg': round(sum(times) / len(times), 4),
                'min': round(min(times), 4),
                'max': round(max(times), 4),
                'std': round(np.std(times), 4),
                'count': len(times),
                'timestamp': datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
        }


### combination 3
### id_in_listings exists

In [66]:
#create index on listings.id
q_create_id_in_listings = '''
CREATE INDEX IF NOT EXISTS id_in_listings
ON listings(id);
'''

#delete the index on reviews.datetime if it exists
q_drop_datetime_in_reviews = '''
DROP INDEX IF EXISTS datetime_in_reviews;
'''


In [67]:
def index_id_in_listings(q, year):

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_create_id_in_listings))
    
    with db_eng.begin() as conn:
        conn.execute(sql_text(q_drop_datetime_index_in_reviews))


    #execute the queries
    with db_eng.begin() as conn:
        result_reviews = conn.execute(sql_text(q_show_indexes))
        print()
        print('The set of indexes on reviews and listings is: ')
        print(result_reviews.all())


    # run it 50 times
    times = []
    for i in range(n):
        start = time.time()
        with db_eng.begin() as conn:
            conn.execute(sql_text(q)).fetchall()
        end = time.time()
        times.append(end - start)

    #store results
    return {
                'avg': round(sum(times) / len(times), 4),
                'min': round(min(times), 4),
                'max': round(max(times), 4),
                'std': round(np.std(times), 4),
                'count': len(times),
                'timestamp': datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
                }


### combination 4
### id_in_listings and datetime_in_reviews exists

In [68]:
#create index on listings.id
q_create_id_in_listings = '''
CREATE INDEX IF NOT EXISTS id_in_listings
ON listings(id);
'''
#create index on reviews.datetime
q_create_datetime_index_in_reviews = '''
CREATE INDEX IF NOT EXISTS datetime_in_reviews
ON reviews(datetime);
'''

In [69]:
def index_both(q, year):

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_create_datetime_index_in_reviews))

    with db_eng.begin() as conn:
        conn.execute(sql_text(q_create_id_in_listings))

    #execute the queries
    with db_eng.begin() as conn:
        result_reviews = conn.execute(sql_text(q_show_indexes))
        print()
        print('The set of indexes on reviews and listings is: ')
        print(result_reviews.all())

    # run it 50 times
    times = []
    for i in range(n):
        start = time.time()
        with db_eng.begin() as conn:
            conn.execute(sql_text(q)).fetchall()
        end = time.time()
        times.append(end - start)

    #store results
    return {
                'avg': round(sum(times) / len(times), 4),
                'min': round(min(times), 4),
                'max': round(max(times), 4),
                'std': round(np.std(times), 4),
                'count': len(times),
                'timestamp': datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
        }



In [70]:
for i in range(2009, 2026):
    year = i
    print(f"Year: {year}")

    # get the query for the year
    q = yyyy_query(year)

    # run the queries with no index
    write_perf_data(i, tags[0], no_index(q, year))

    # run the queries with index on reviews.datetime
    write_perf_data(i, tags[1], index_datetime_in_reviews(q, year))

    # run the queries with index on listings.id
    write_perf_data(i, tags[2], index_id_in_listings(q, year))

    # run the queries with both indexes
    write_perf_data(i, tags[3], index_both(q, year))

    print("Finished all combinations for year {year}.")


Year: 2009

The set of indexes on reviews and listings is: 
[('airbnb', 'reviews', 'comments_tsv_in_reviews', None, 'CREATE INDEX comments_tsv_in_reviews ON airbnb.reviews USING gin (comments_tsv)'), ('airbnb', 'listings', 'neigh_in_listings', None, 'CREATE INDEX neigh_in_listings ON airbnb.listings USING btree (neighbourhood_cleansed)'), ('airbnb', 'listings', 'neigh_groups_in_listings', None, 'CREATE INDEX neigh_groups_in_listings ON airbnb.listings USING btree (neighbourhood_group_cleansed)')]
successfully wrote __ 2009 to listings_join_reviews.json



The set of indexes on reviews and listings is: 
[('airbnb', 'reviews', 'comments_tsv_in_reviews', None, 'CREATE INDEX comments_tsv_in_reviews ON airbnb.reviews USING gin (comments_tsv)'), ('airbnb', 'listings', 'neigh_in_listings', None, 'CREATE INDEX neigh_in_listings ON airbnb.listings USING btree (neighbourhood_cleansed)'), ('airbnb', 'listings', 'neigh_groups_in_listings', None, 'CREATE INDEX neigh_groups_in_listings ON airbnb.lis